In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as plt
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Point 1: 
Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
loan_lenders = pd.read_csv('data/loans_lenders.csv')

In [3]:
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
def normalize(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [5]:
%%time

norm_loan_lenders = normalize(loan_lenders, 'lenders', ',')

CPU times: user 39.7 s, sys: 2.71 s, total: 42.4 s
Wall time: 42.5 s


In [6]:
norm_loan_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


### Point 2: 
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [7]:
%%time

loans = pd.read_csv('data/loans.csv')[['loan_id', 'country_code', 'loan_amount', 'num_lenders_total', 'disburse_time', 'planned_expiration_time']]

CPU times: user 52.5 s, sys: 7.36 s, total: 59.8 s
Wall time: 1min 4s


a lot of nan's!

In [8]:
print(len(loans.planned_expiration_time[loans.planned_expiration_time.isnull()]),
len(loans.disburse_time[loans.disburse_time.isnull()]))

371834 2813


let's convert to datetime all not nan values of disburse_time and planned_expiration_time;
#### note:
nan values in a pandas series are represented as float

In [9]:
formatter = '%Y-%m-%d %H:%M:%S.%f +0000'

In [10]:
%%time 

# TODO: find a more efficient way to do that, if any..

loans['disburse_time'] = loans['disburse_time'].apply(lambda str_date: datetime.datetime.strptime(str_date, formatter) if not (isinstance(str_date, float) and np.isnan(str_date)) else str_date)
loans['planned_expiration_time'] = loans['planned_expiration_time'].apply(lambda str_date: datetime.datetime.strptime(str_date, formatter) if not (isinstance(str_date, float) and np.isnan(str_date)) else str_date)

CPU times: user 1min 12s, sys: 1.33 ms, total: 1min 12s
Wall time: 1min 12s


In [11]:
%%time

loans['duration'] = loans.planned_expiration_time - loans.disburse_time

CPU times: user 23.5 ms, sys: 215 µs, total: 23.7 ms
Wall time: 23.7 ms


In [12]:
loans.duration

0         53 days 19:30:06
1         96 days 14:25:07
2         37 days 13:10:05
3         34 days 19:10:02
4         57 days 22:10:02
                ...       
1419602   39 days 17:00:03
1419603   39 days 08:40:07
1419604   51 days 14:20:04
1419605   63 days 00:50:02
1419606   61 days 12:50:06
Name: duration, Length: 1419607, dtype: timedelta64[ns]

There are 14935 loans which duration is negative: it means that the deadline was not satisfied

In [13]:
loans.duration[loans.duration < pd.Timedelta(0)]

63        -31 days +21:40:03
96        -25 days +06:30:11
177       -25 days +21:00:03
207       -33 days +06:30:08
217       -31 days +17:40:03
                 ...        
1419303   -35 days +21:30:03
1419304   -32 days +17:50:04
1419374   -34 days +21:40:03
1419443   -30 days +06:00:03
1419581   -30 days +16:50:04
Name: duration, Length: 14935, dtype: timedelta64[ns]

### Point 3: Find the lenders that have funded at least twice

In [14]:
%%time

loans_by_lenders = norm_loan_lenders.groupby('lenders')['loan_id'].count().reset_index(name='fund_count')

CPU times: user 17.5 s, sys: 0 ns, total: 17.5 s
Wall time: 17.5 s


In [15]:
loans_by_lenders[loans_by_lenders.fund_count > 1].head(15)

,lenders,fund_count
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
6,0154884,4
7,0161130,2
8,0169713,3
9,0185429,2
11,0206338,2


### Point 4: For each country, compute how many loans have involved that country as borrowers

### Point 5: For each country, compute the overall amount of money borrowed

### Point 6: Like the previous point, but expressed as a percentage of the overall amount lent

In [16]:
def borrow_summary(dimension: str) -> pd.DataFrame:
    grouped = loans.groupby(dimension)
    borrow_summary = grouped['loan_id'].count().reset_index(name='borrow_count')
    borrow_summary['overall_amount_borrowed'] = grouped['loan_amount'].sum().reset_index()['loan_amount']
    overall_amount_lent = loans['loan_amount'].sum()
    borrow_summary['overall_amount_borrowed/total'] = (borrow_summary['overall_amount_borrowed'] / overall_amount_lent) * 100
    return borrow_summary

In [17]:
borrow_by_country = borrow_summary('country_code')

In [18]:
borrow_by_country.head(15)

,country_code,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
0,AF,2337,1967950.00000,0.16657
1,AL,3075,4307350.00000,0.36459
2,AM,13952,22950475.00000,1.94259
3,AZ,10172,14784625.00000,1.25141
4,BA,608,477250.00000,0.04040
5,BF,3489,4085200.00000,0.34578
6,BG,296,375300.00000,0.03177
7,BI,1727,5233450.00000,0.44297
8,BJ,5946,3865825.00000,0.32721
9,BO,25250,44226725.00000,3.74347


In [19]:
borrow_by_country.describe()

,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
count,95.00000,95.00000,95.00000
mean,14943.13684,12435766.05263,1.05260
std,35149.88448,18263180.36163,1.54584
min,1.00000,5000.00000,0.00042
25%,545.00000,752525.00000,0.06370
50%,4681.00000,4181100.00000,0.35390
75%,15117.00000,16728425.00000,1.41594
max,285336.00000,97984600.00000,8.29368


### Point 7: Like the three previous points, but split for each year (with respect to disburse_time)

In [20]:
%%time

loans['disburse_year'] = loans['disburse_time'].map(lambda t: t.year)

CPU times: user 11.3 s, sys: 88.1 ms, total: 11.4 s
Wall time: 11.4 s


In [21]:
borrow_by_year = borrow_summary('disburse_year')

In [22]:
borrow_by_year

,disburse_year,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
0,2005.00000,203,102850.00000,0.00871
1,2006.00000,2172,1376575.00000,0.11652
2,2007.00000,24400,15446525.00000,1.30744
3,2008.00000,54586,39423050.00000,3.33687
4,2009.00000,83076,59689475.00000,5.05228
5,2010.00000,93466,72609150.00000,6.14583
6,2011.00000,114540,93699300.00000,7.93096
7,2012.00000,133650,119977575.00000,10.15522
8,2013.00000,140167,132043925.00000,11.17655
9,2014.00000,172709,152270425.00000,12.88857


### Point 8: For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [23]:
loans['base_contribution'] = loans.loan_amount / loans.num_lenders_total

In [24]:
ids = set(loans.loan_id) - set(loan_lenders.loan_id)
ids2 = set(loan_lenders.loan_id) - set(loans.loan_id)


In [25]:
loans_cleaned = loans[(loans.num_lenders_total > 0) & (~loans.loan_id.isin(ids))] ### verify maybe useless

In [26]:
norm_cleaned = norm_loan_lenders[~norm_loan_lenders.loan_id.isin(ids2)] ### verify maybe useless

In [27]:
%%time

joined = pd.concat([norm_cleaned.set_index('loan_id'), loans_cleaned.set_index('loan_id')], axis=1, join='inner')

CPU times: user 2.89 s, sys: 3.2 s, total: 6.09 s
Wall time: 6.12 s


In [28]:
overall_amount_per_lender = joined.groupby('lenders')['base_contribution'].sum().reset_index(name='overall_amount_lent')

In [29]:
overall_amount_per_lender.head(15)

,lenders,overall_amount_lent
0,000,1485.30966
1,00000,1249.94736
2,0002,2201.18046
3,00mike00,38.46154
4,0101craign0101,2424.08893
5,0132575,113.58770
6,0154884,122.07768
7,0161130,52.73973
8,0169713,90.68987
9,0185429,52.86378


In [30]:
overall_amount_per_lender.tail(15)

,lenders,overall_amount_lent
1639011,zvi,370.57695
1639012,zvi1263,28.84615
1639013,zvika5974,35.71429
1639014,zvonimir7460,34.37500
1639015,zx147,177.17835
1639016,zyra9641,26.88679
1639017,zyrah8525,166.69643
1639018,zyrorl,92.59959
1639019,zzaba,25.00000
1639020,zzaman,642.25094


In [31]:
overall_amount_per_lender.describe()

,overall_amount_lent
count,1639026.00000
mean,637.10046
std,12587.00054
min,5.43478
25%,33.48962
50%,88.17220
75%,289.14772
max,6296800.48566


### Point 9: For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [32]:
borrow_by_country.head()

,country_code,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total
0,AF,2337,1967950.00000,0.16657
1,AL,3075,4307350.00000,0.36459
2,AM,13952,22950475.00000,1.94259
3,AZ,10172,14784625.00000,1.25141
4,BA,608,477250.00000,0.04040


In [33]:
lenders = pd.read_csv('data/lenders.csv')

In [34]:
sum(lenders['country_code'].isna()) ### lot of nan's. we must assume 

1458635

In [35]:
lenders['country_code'].dropna().value_counts()

US    591612
CA     67970
GB     38380
AU     37103
DE     16007
       ...  
GW         1
MS         1
KM         1
IO         1
TF         1
Name: country_code, Length: 234, dtype: int64

In [36]:
lenders['country_code'].dropna().describe()

count     890539
unique       234
top           US
freq      591612
Name: country_code, dtype: object

In [37]:
s = lenders.country_code.value_counts(normalize=True)

In [38]:
missing = lenders['country_code'].isnull()
lenders.loc[missing,'country_code'] = np.random.choice(s.index, size=len(lenders[missing]),p=s.values)

In [39]:
overall_amount_per_lender.lenders = overall_amount_per_lender.lenders.apply(lambda x: x.strip())

In [40]:
%%time

df4 = pd.concat([overall_amount_per_lender.set_index('lenders'), lenders.set_index('permanent_name')], axis=1, join='inner')

CPU times: user 9.88 s, sys: 11.3 ms, total: 9.89 s
Wall time: 9.9 s


In [44]:
lend_by_country = df4.groupby('country_code')['overall_amount_lent'].sum().reset_index(name='overall_amount_lent')

In [46]:
lend_by_country.head()

,country_code,overall_amount_lent
0,AD,11946.35850
1,AE,1763811.46252
2,AF,140235.43471
3,AG,2340.00991
4,AI,322.57733


In [48]:
len(loans['country_code'].unique())

96

In [49]:
len(lenders['country_code'].unique())

234

In [57]:
overall_in_out = pd.concat([borrow_by_country.set_index('country_code'), lend_by_country.set_index('country_code')], axis = 1, join='inner')

In [59]:
overall_in_out['lent - borrow'] = overall_in_out['overall_amount_lent'] - overall_in_out['overall_amount_borrowed']

In [62]:
overall_in_out.tail(15)

,borrow_count,overall_amount_borrowed,overall_amount_borrowed/total,overall_amount_lent,lent - borrow
country_code,,,,,
TZ,16110,18720300.00000,1.58454,70367.01870,-18649932.98130
UA,4994,7091800.00000,0.60027,114280.46340,-6977519.53660
UG,45882,38386500.00000,3.24914,102381.23528,-38284118.76472
US,9180,46352000.00000,3.92336,631675518.47026,585323518.47026
UY,1,8000.00000,0.00068,105222.22637,97222.22637
VC,16,49225.00000,0.00417,6786.12962,-42438.87038
VI,2,10000.00000,0.00085,21853.39804,11853.39804
VN,21839,24681100.00000,2.08907,425110.09003,-24255989.90997
VU,4,9250.00000,0.00078,295842.49563,286592.49563
